In [16]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
import sqlite3
import numpy as np

In [14]:
conn = sqlite3.connect("../tracks.db")
df = pd.read_sql_query("SELECT * FROM tracks", conn)

In [15]:
df

,song,artist,genre,year,styles,country,albumCover
0,Blinded By Science,Thomas Dolby,"[""Electronic"", ""Rock""]",1983,"[""Synth-pop"", ""New Wave""]",US,https://i.discogs.com/gl6sCk30HMBaT4R3vafPT8-p...
1,Dream Into Action,Howard Jones,"[""Electronic"", ""Pop""]",1985,"[""Synth-pop""]",US,https://i.discogs.com/kPrYw5FODCnojTR5inuLw6B9...
2,Locked Down,Dr. John,"[""Rock"", ""Funk / Soul"", ""Blues""]",2012,"[""Louisiana Blues"", ""Bayou Funk"", ""Psychedelic""]",US,https://i.discogs.com/_Lb88gYRnfezPEKouZnvuDts...
3,Hold On,Romanthony,"[""Electronic""]",1999,"[""House""]",France,https://i.discogs.com/sW7FHhRVr1n8PLaEIgWX6Zcv...
4,Ni - Ten - Ichi - Ryu,Photek,"[""Electronic""]",1997,"[""Drum n Bass""]",UK,https://i.discogs.com/U-FjpZ7ifurLAvOe7emsEvWW...
...,...,...,...,...,...,...,...
634,The Electric Institute Sampler,Various,"[""Electronic""]",2005,"[""House"", ""Broken Beat"", ""Techno""]",UK,https://i.discogs.com/EWGw25xpdNpoKRpncUrGflgL...
635,Lift Every Voice (Take Me Away),Jasper Street Co.,"[""Electronic""]",2000,"[""House"", ""Garage House""]",US,https://i.discogs.com/9NILOQIcU6nZgckgLxa7S6wf...
636,Daydream,Wallace Collection,"[""Rock""]",1969,"[""Symphonic Rock""]",France,https://i.discogs.com/1mn3gffT0s1eKVJe4730yT57...
637,The Chris Tetley Interviews,Metallica,"[""Non-Music""]",1987,"[""Interview""]",UK,https://i.discogs.com/Y6OFvfwA5Fv2R7piZY_ESZXo...


In [17]:
df_encoded = df.copy()

# Label Encoding for artist, genre, styles, country
label_encoder = LabelEncoder()
df_encoded['artist'] = label_encoder.fit_transform(df_encoded['artist'])
df_encoded['genre'] = label_encoder.fit_transform(df_encoded['genre'])
df_encoded['styles'] = label_encoder.fit_transform(df_encoded['styles'])
df_encoded['country'] = label_encoder.fit_transform(df_encoded['country'])

# Scale Year
scaler = StandardScaler()
df_encoded['year_scaled'] = scaler.fit_transform(df_encoded[['year']])

# One-Hot Encoding (if needed)
onehot_encoder = OneHotEncoder()
encoded_features = onehot_encoder.fit_transform(df_encoded[['genre', 'styles', 'country']]).toarray()

# Combine Scaled Year with Encoded Features
final_features = np.hstack((encoded_features, df_encoded[['year_scaled']].values))


In [18]:
# Calculate cosine similarity with the updated features
similarity_matrix = cosine_similarity(final_features)


In [22]:
user_preferences = {}  # Dictionary to store user preferences

def update_user_preferences(user_id, song_name, like_dislike):
    if user_id not in user_preferences:
        user_preferences[user_id] = {'likes': [], 'dislikes': []}

    if like_dislike == 'like':
        user_preferences[user_id]['likes'].append(song_name)
    elif like_dislike == 'dislike':
        user_preferences[user_id]['dislikes'].append(song_name)


In [23]:
def recommend_based_on_preferences(user_id, top_n):
    if user_id not in user_preferences:
        print("No user preferences found. Recommending general top songs.")
        # Implement logic to recommend general top songs
        return

    liked_songs = user_preferences[user_id]['likes']
    disliked_songs = user_preferences[user_id]['dislikes']

    recommendations = []
    for song in liked_songs:
        if song in df['song'].values:
            song_index = df[df['song'] == song].index[0]
            similar_songs = sorted(list(enumerate(similarity_matrix[song_index])), key=lambda x: x[1], reverse=True)[1:top_n+1]
            for i, score in similar_songs:
                recommended_song = df.iloc[i]['song']
                if recommended_song not in liked_songs and recommended_song not in disliked_songs:
                    recommendations.append(recommended_song)
                    if len(recommendations) >= top_n:
                        break
            if len(recommendations) >= top_n:
                break

    print(f"Recommendations for user {user_id}: {recommendations}")


In [21]:
user_id = "user123"
update_user_preferences(user_id, "Liked Song Title", 'like')
update_user_preferences(user_id, "Disliked Song Title", 'dislike')


Recommendations for Locked Down:
Energy
GAMI GANG
Cowboy Bebop (Original Series Soundtrack)
Undress
Don't Let's Start


In [ ]:
recommend_based_on_preferences(user_id, 5)